In [14]:
import spacy
import pandas as pd
import numpy as np
import pickle
import random
import nltkb
from pprint import pprint

In [28]:
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA

In [34]:
dir(spacy.attrs)
dir(POS)

['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes']

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
doc = nlp("The quick brown fox jumps over the lazy dog.")

In [5]:
for tok in doc:
    print(tok.text, tok.pos_, tok.dep_)

The DET det
quick ADJ amod
brown ADJ amod
fox NOUN compound
jumps NOUN ROOT
over ADP prep
the DET det
lazy ADJ amod
dog NOUN pobj
. PUNCT punct


In [6]:
for tok in doc.noun_chunks:
    print(tok)

The quick brown fox jumps
the lazy dog


In [7]:
review_data = None
with open("ICLR data/papers_2020_balanced.pickle", "rb") as f:
    review_data = pickle.load(f)

In [9]:
list(review_data.keys())[0:10]

['H1gNOeHKPS',
 'S1xtORNFwH',
 'SJx9ngStPH',
 'BklOXeBFDS',
 'ByeGzlrKwH',
 'rJeGJaEtPH',
 'SJldu6EtDS',
 'BJe-unNYPr',
 'BkgF4kSFPB',
 'SJeUm1HtDH']

In [19]:
pprint(review_data["H1gNOeHKPS"]["OFF_REVIEW"][0]["content"]["review"])

('The authors propose the Neural Multiplication Unit (NMU), which can learn to '
 'solve a family of arithmetic operations using -, + and * atomic operations '
 'over real numbers from examples. They show that a combination of careful '
 'initialization, regularization and structural choices allows their model to '
 'learn more reliably and efficiently than the previously published Neural '
 'Arithmetic Logic Unit.\n'
 '\n'
 'The NALU consists of two additive sub-units in the real and log-space '
 'respectively, which allows it to handle both additions/subtractions and '
 'multiplications/divisions, and combines them with a gating mechanism. The '
 'NMU on the other hand simply learns a product of affine transformations of '
 'the input. This choice prevents the model from learning divisions, which the '
 'authors argue made learning unstable for the NALU case, but allows for an a '
 'priori better initialization and dispenses with the gating which is '
 'empirically hard to learn. The

In [20]:
doc = nlp(review_data["H1gNOeHKPS"]["OFF_REVIEW"][0]["content"]["review"])

In [47]:
for tok in doc.noun_chunks:
    print(tok)

The authors
the Neural Multiplication Unit
NMU
a family
arithmetic operations
and * atomic operations
real numbers
examples
They
a combination
careful initialization
regularization
structural choices
their model
the previously published Neural Arithmetic Logic Unit
The NALU
two additive sub-units
the real and log-space
it
both additions/subtractions
multiplications
/divisions
them
a gating mechanism
The NMU
the other hand
a product
affine transformations
the input
This choice
the model
learning divisions
the authors
the NALU case
an a priori better initialization
dispenses
the gating
The departures
the NALU architecture
significant improvements
the considered applications
extrapolation
the training domain
The paper
one notable exception
the form
the loss function
the paper
the scope
the work
the authors
the application
it
a wider context
divisions
a sufficient justification
them
the authors
future work
the proposed unit
at least 10K examples
a single expression
fewer than 10 inputs
the

In [58]:
for i in range(len(doc)):
    if doc[i].pos_ == "ADJ" and doc[i+1].tag_.startswith("NN"):
        print(doc[i-1], doc[i-1].tag_, doc[i], doc[i+1])

of IN arithmetic operations
* NFP atomic operations
over IN real numbers
of IN careful initialization
and CC structural choices
two CD additive sub
the DT other hand
of IN affine transformations
priori FW better initialization
to IN significant improvements
one CD notable exception
a DT wider context
a DT sufficient justification
a DT plausible way
for IN future work
a DT single expression
or CC efficient architecture


The
authors
propose
the
Neural
Multiplication
Unit
(
NMU
)
,
which
can
learn
to
solve
a
family
of
arithmetic
operations
using
-
,
+
and
*
atomic
operations
over
real
numbers
from
examples
.
They
show
that
a
combination
of
careful
initialization
,
regularization
and
structural
choices
allows
their
model
to
learn
more
reliably
and
efficiently
than
the
previously
published
Neural
Arithmetic
Logic
Unit
.



The
NALU
consists
of
two
additive
sub
-
units
in
the
real
and
log
-
space
respectively
,
which
allows
it
to
handle
both
additions
/
subtractions
and
multiplications
/
divisions
,
and
combines
them
with
a
gating
mechanism
.
The
NMU
on
the
other
hand
simply
learns
a
product
of
affine
transformations
of
the
input
.
This
choice
prevents
the
model
from
learning
divisions
,
which
the
authors
argue
made
learning
unstable
for
the
NALU
case
,
but
allows
for
an
a
priori
better
initialization
and
dispenses
with
the
gating
which
is
empirically
hard
to
learn
.
The
departures
from
the
NALU
architectu

In [22]:
dir(doc)

['_', '__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '_bulk_merge', '_py_tokens', '_realloc', '_vector', '_vector_norm', 'cats', 'char_span', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_disk', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'is_nered', 'is_parsed', 'is_sentenced', 'is_tagged', 'lang', 'lang_', 'mem', 'merge', 'noun_chunks', 'noun_chunks_iterator', 'print_tree', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_extension', 'similarity', 'tensor', 'text', 'text_with_ws', 'to_array', 'to_bytes', 'to_disk', 'to_json', 'to_utf8_array', 'user_data', 'user_hooks', 'user_span_hooks', 'u

In [31]:
nparr = doc.to_array([LOWER, POS, ENT_TYPE, IS_ALPHA])

In [38]:
for t in nparr:
    if t[1] == "ADJ"
#     print((t.view()))
#     break

[7425985699627899538                  90                   0
                   1]


TypeError: 'int' object is not callable